## 1. 加载并清洗数据
- 只需要跑一次就行

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk

In [2]:
# 先将原始文件存储为csv文件，方便处理

df = pd.read_csv('PatentsAll.csv')
print(df.shape) # 查看行列数
df.head() # 前几列数据

(81957, 13)


,Unnamed: 0,Publication number,Live,Expired,Withdrawn/Abandoned,Priority date,Current direct owner(s) name(s),Current direct owner(s) country code(s),Current direct owner(s) BvD ID Number(s),IPC code (main),IPC code label (main),Title,Abstract
0,1,BG108387A,Yes,NaN,NaN,27/11/2003,ILCHEV BLAGOJ,PL,NaN,H02K7/00,Arrangements for handling mechanical energy st...,ENGINE DESIGNED ON THE PRINCIPLE OF WORM GEAR ...,Двигателят може да намери приложение като меха...
1,2,CN2062130U,NaN,Yes,NaN,20/12/1989,MO YANING,MY,NaN,H02K21/22,Synchronous motors having permanent magnets; S...,OUTER ROTARY PERMANENT MAGNET A.C. GENERATOR,The utility model provides an outer rotary per...
2,3,CN206099691U,Yes,NaN,NaN,26/10/2016,XI'AN UNIV OF SCIENCE AND TECH,RO,NaN,H02K16/00,Machines with more than one rotor or stator,PERMANENT MAGNET SYNCHRONOUS MOTOR,The utility model discloses a permanent magnet...
3,4,EP1946430A1,Yes,NaN,NaN,19/10/2005,"Zepp, Lawrence P.",NaN,NaN,H02K21/00,Synchronous motors having permanent magnets; S...,BRUSHLESS PERMANENT MAGNET MOTOR/ GENERATOR WI...,A permanent magnet motor/generator that includ...
4,5,CN102263444A,NaN,Yes,NaN,02/08/2011,刘勤,RO,NaN,H02K1/12,Details of the magnetic circuit (magnetic circ...,WITH A PERMANENT MAGNET AND AN ELECTROMAGNET T...,The invention discloses a disc type magnetic m...


In [6]:
# 只保留需要的几列
col = ['Publication number', 'Title', 'Abstract']
df = df[col]
df = df[pd.notnull(df['Publication number'])]
df = df[pd.notnull(df['Title'])]
df = df[pd.notnull(df['Abstract'])]
df.columns = ['Publication number', 'Title', 'Abstract']
# 修改大小写
df['Title'] = df['Title'].str.title().str.replace('Ndfeb', 'NdFeB')
df['Abstract'] = df['Abstract'].str.capitalize().str.replace('Ndfeb', 'NdFeB').str.replace('ndfeb', 'NdFeB') 
df.head()
print(df.shape)
df.head()

(75710, 3)


,Publication number,Title,Abstract
0,BG108387A,Engine Designed On The Principle Of Worm Gear ...,Двигателят може да намери приложение като меха...
1,CN2062130U,Outer Rotary Permanent Magnet A.C. Generator,The utility model provides an outer rotary per...
2,CN206099691U,Permanent Magnet Synchronous Motor,The utility model discloses a permanent magnet...
3,EP1946430A1,Brushless Permanent Magnet Motor/ Generator Wi...,A permanent magnet motor/generator that includ...
4,CN102263444A,With A Permanent Magnet And An Electromagnet T...,The invention discloses a disc type magnetic m...


## 2. 切片并翻译
- 由于谷歌翻译api的访问限制，这一步需要根据实际情况调节range()的大小和范围，少量多次
- 结果输出了从a行到b行，共花了多长时间
- 每一次运行都是在原数据的基础上运行的，后面的输出包括了前面的输出结果
- 为了防止出错，可以适当进行保存并输出

In [47]:
from google_trans_new import google_translator
from multiprocessing.dummy import Pool as ThreadPool
import time

pool = ThreadPool(30)
detector = google_translator(timeout=10)  
translator = google_translator(timeout=10)
time1 = time.time()
for j in range(8500,9000): # range(a,b),遍历a-b行
    df.iloc[j,1] = translator.translate(df.iloc[j,1], lang_tgt='en') # 翻译Title
    df.iloc[j,2] = translator.translate(df.iloc[j,2], lang_tgt='en') # 翻译Abstract     
pool.close()
pool.join()

time2 = time.time() 
print("Translated, a total of %s s."%(time2 - time1))

# 如果报错了，就缩短a,b的距离，重新跑这一块儿就行

google_new_transError: Failed to connect. Probable cause: timeout

In [37]:
# 保存翻译结果，可以先翻译完，再进行分类的操作，不需要每次都运行，可以在适当环节保存，在运行结束时保存
outputpath='translate0-8000.csv' # 输出路径，本地文件夹路径也可以
df.to_csv(outputpath,sep=',',index=False,header=True)
# 同时，翻译完后，还是应该先看看翻译的结果，有没有乱码，检查稍微手动修改后，再做分类处理

## 3. 分类操作
- 写在另一份文档里面

In [50]:
df2 = df[20000:40000]
outputpath='translate0-20000-40000.csv' # 输出路径，本地文件夹路径也可以
df2.to_csv(outputpath,sep=',',index=False,header=True)